### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

5


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
150,buradaydı,16726
151,buysa,16063
152,miydiniz,15468
153,bunlarla,15042


In [9]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [10]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,dutch_word
0,kanser,kanker
1,viski,whisky
2,litre,liter
3,gitar,gitaar
4,tren,trein
...,...,...
1563,zebra,zebra
1564,zikzak,zigzag
1565,zombi,zombie
1566,zooloji,zoölogie


In [11]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,dutch_word


In [12]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [13]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [14]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [15]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}{file_ext}.xlsx", index=False)    

#### Concat Result

In [16]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,birden
1,burda
2,mı
3,senden
4,o
...,...
150,değilim
151,varmış
152,neler
153,değildi


In [17]:
# Option For Frequency
if shared_word_frequency:
    df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
150,buradaydı,16726
151,buysa,16063
152,miydiniz,15468
153,bunlarla,15042


In [18]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram{file_ext}.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
6870,nedir biraz,6
6871,mı bunlara,6
6872,ne nesi,6
6873,nesin bir,6


In [19]:
# Option For Twogram Sentence
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram


,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
6870,nedir biraz,6
6871,mı bunlara,6
6872,ne nesi,6
6873,nesin bir,6


In [20]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram{file_ext}.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
16922,ama o sendin,10
16923,varsa da ne,10
16924,varsa da buna,10
16925,varsa da bana,10


In [21]:
# Option For Threegram Sentence
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_twogram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
16922,ama o sendin,10
16923,varsa da ne,10
16924,varsa da buna,10
16925,varsa da bana,10


In [22]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram{file_ext}.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,bir şey var mı,41773
1,bir şey daha var,12185
2,bir şey mi var,11834
3,ne var ne yok,9276
4,bir şey değil mi,5080
...,...,...
9169,mi seni ne kadar,7
9170,benimle daha ne kadar,7
9171,biraz şey gibi biraz,7
9172,biraz şey değiller mi,7


In [23]:
# Option For Fourgram Sentence
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram_sent = df_twogram_sent.loc[:,["fourgram"]]
    df_shared_fourgram = pd.merge(df_shared_fourgram, df_fourgram_sent, how="inner", on="fourgram")
    df_shared_fourgram.drop_duplicates(inplace=True)
    df_shared_fourgram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fourgram

,fourgram,freq_fourgram
0,bir şey var mı,41773
1,bir şey daha var,12185
2,bir şey mi var,11834
3,ne var ne yok,9276
4,bir şey değil mi,5080
...,...,...
9169,mi seni ne kadar,7
9170,benimle daha ne kadar,7
9171,biraz şey gibi biraz,7
9172,biraz şey değiller mi,7


In [24]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram{file_ext}.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,senin için bir şeyim var,2283
1,evet onun gibi bir şey,1046
2,bir şey varsa o da,1010
3,bir şey yok değil mi,893
4,o kadar çok şey var,790
...,...,...
1271,bir şey vardı ama şey,10
1272,bir şey var ve senin,10
1273,bir şey var mı bu,10
1274,bir şey var mı sende,10


In [25]:
# Option For Fivegram Sentence
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram_sent = df_twogram_sent.loc[:,["fivegram"]]
    df_shared_fivegram = pd.merge(df_shared_fivegram, df_fivegram_sent, how="inner", on="fivegram")
    df_shared_fivegram.drop_duplicates(inplace=True)
    df_shared_fivegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fivegram

,fivegram,freq_fivegram
0,senin için bir şeyim var,2283
1,evet onun gibi bir şey,1046
2,bir şey varsa o da,1010
3,bir şey yok değil mi,893
4,o kadar çok şey var,790
...,...,...
1271,bir şey vardı ama şey,10
1272,bir şey var ve senin,10
1273,bir şey var mı bu,10
1274,bir şey var mı sende,10


In [26]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence{file_ext}.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,ne var,62532
1,ben de,59972
2,değil mi,58386
3,bu da ne,50098
4,ben mi,33652
...,...,...
5343,o da ne o da ne,6
5344,hayır ben şeye,6
5345,ve ben de ondan,6
5346,ne neyin var,6


In [27]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bir,18835735.0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,senin için bir şeyim var,2283.0,ne var,62532.0
1,bu,11062659.0,değil mi,585879.0,bir şey var,110455,bir şey daha var,12185.0,evet onun gibi bir şey,1046.0,ben de,59972.0
2,ne,8025880.0,ben de,377765.0,bu da ne,89463,bir şey mi var,11834.0,bir şey varsa o da,1010.0,değil mi,58386.0
3,ve,7766036.0,var mı,281902.0,bir şey değil,68498,ne var ne yok,9276.0,bir şey yok değil mi,893.0,bu da ne,50098.0
4,için,5484109.0,ne kadar,237876.0,bir şey mi,64933,bir şey değil mi,5080.0,o kadar çok şey var,790.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16922,NaN,NaN,NaN,NaN,ama o sendin,10,NaN,NaN,NaN,NaN,NaN,NaN
16923,NaN,NaN,NaN,NaN,varsa da ne,10,NaN,NaN,NaN,NaN,NaN,NaN
16924,NaN,NaN,NaN,NaN,varsa da buna,10,NaN,NaN,NaN,NaN,NaN,NaN
16925,NaN,NaN,NaN,NaN,varsa da bana,10,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx", index=False)

##### Concat Result With Comma

In [29]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [30]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [31]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [32]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [33]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,ama,"ama bu, ama ben, evet ama, ama o, ama sen, ama...","çok ama çok, ama bir şey, ama değil mi, evet a...","ama o kadar da, bir şey değil ama, bir şey yok...","ama bir şey daha var, ama o kadar da değil, am...","evet ama, ama ben, ama ne, ama bu, ama sen, ha..."
1,bana,"bana bir, bana ne, bunu bana, onu bana, bana d...","bana bir şey, sen de bana, bir de bana, bana b...","bana bir şey mi, ve sen de bana, evet bir de b...","bana bir şey var mı, bana da biraz var mı, sen...","bana mı, bana ne, bana da, bana değil, bu bana..."
2,ben,"ben de, ben bir, ve ben, ben mi, ama ben, hayı...","ben de seni, sen ve ben, ve ben de, evet ben d...","ben de senin gibi, ben senin gibi değilim, ben...","ne sen ne de ben, ben de senin gibi bir, sen m...","ben de, ben mi, hayır ben, evet ben de, ben de..."
3,bence,"bence de, bence bu, bence çok, evet bence, ben...","evet bence de, bence bu çok, bence sen de, ben...","bence bu senin için, bence o kadar da, ve benc...","bence bu senin için çok, bir şey yok ama bence...","bence de, evet bence de, bence değil, bence va..."
4,bende,"bende değil, bende de, bende var, bende bir, b...","bende de var, bende bir şey, bende ne var, ben...","bende bir şey yok, bende bir şey var, bir şey ...","bende de bir şey yok, bende senin için bir şey...","bende değil, bende var, evet bende, bende yok,..."
...,...,...,...,...,...,...
150,şeyler,"bir şeyler, şeyler var, şeyler de, gibi şeyler...","bir şeyler var, için bir şeyler, bir şeyler mi...","bir şeyler var mı, bir şeyler mi var, ben de b...","bir şeyler var değil mi, benim için de bir şey...","ne gibi şeyler, bir şeyler, bunun gibi şeyler,..."
151,şeylerden,"şeylerden biri, bir şeylerden, bu şeylerden, o...","şeylerden biri de, o şeylerden biri, şeylerden...","şeylerden biri de bu, şeylerden biri değil mi,...","ben o şeylerden biri değilim, gibi şeylerden b...","ben o şeylerden biri değilim, bu o şeylerden, ..."
152,şeylere,"bir şeylere, bu şeylere, şeylere ne, o şeylere...","için bir şeylere, bu gibi şeylere, bir şeylere...","bir şeylere bu kadar, bu şeylere ne kadar, ben...",NaN,bu şeylere
153,şeyleri,"bir şeyleri, bu şeyleri, o şeyleri, şeyleri yo...","bir şeyleri yok, bir şeyleri var, bu gibi şeyl...","bir şeyleri var mı, ben de bir şeyleri, bir şe...",sence ben bu gibi şeyleri,"bir şeyleri yok, ne gibi şeyleri, ne şeyleri, ..."


In [34]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bir,18835735,"bir şey, bu bir, bir şeyler, bir daha, ben bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","senin için bir şeyim var, evet onun gibi bir ş...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, bu ben...","bu da ne, bu kadar mı, bu senin için, bu da ne...","bu da neyin nesi, bu benim için çok, bu şey de...","bu da bir şey mi, bu daha bir şey değil, ve bu...","bu da ne, bu ne, nedir bu, bu nedir, bu o, bu ..."
2,ne,8025880,"ne kadar, ne var, da ne, bu ne, burada ne, sen...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, senin için ne kadar, bu şey de ...","sende ne var ne yok, bunun senin için ne kadar...","ne var, bu da ne, ne için, bu ne, ne kadar, ne..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, sen ve ben birlikte, ve sen d...","ve bir şey daha var, ve bu da senin için, ve b...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b..."
4,için,5484109,"senin için, benim için, için bir, bunun için, ...","benim için bir, senin için bir, bu senin için,...","benim için bir şey, senin için bir şey, bu ben...","senin için bir şeyim var, senin için bir şey v...","ne için, senin için, bu senin için, benim için..."
...,...,...,...,...,...,...,...
150,buradaydı,16726,"o buradaydı, da buradaydı, buradaydı ve, de bu...","buradaydı değil mi, o buradaydı ve, o da burad...","buradaydı ve senden daha, de buradaydı değil mi",NaN,"o buradaydı, evet buradaydı, hayır buradaydı, ..."
151,buysa,16063,"şey buysa, buysa bunu, buysa ben, buysa o, buy...","buysa ben de, buysa benim için, şey buysa bunu...","buysa seni bir daha, buysa bunu ben de, şey bu...",NaN,NaN
152,miydiniz,15468,"değil miydiniz, birlikte miydiniz, miydiniz ha...","burada değil miydiniz, onunla birlikte miydini...",NaN,NaN,"birlikte miydiniz, birlikte değil miydiniz, de..."
153,bunlarla,15042,"bunlarla ne, bunlarla bir, bunlarla mı, onun b...","benim bunlarla bir, onun bunlarla ne, onun bun...",o da beni bunlarla,NaN,"bunlarla mı, bunlarla değil, bunlarla birlikte"


In [35]:
df_word_order_join_all.word.nunique()

155

In [36]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [37]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}.xlsx")
output_file

['Turkish_Dutch_Shared_Twogram5.xlsx',
 'Turkish_Dutch_Shared_Threegram5.xlsx',
 'Turkish_Dutch_Shared_Fourgram5.xlsx',
 'Turkish_Dutch_Shared_Fivegram5.xlsx',
 'Turkish_Dutch_Shared_Sentence5.xlsx',
 'Turkish_Dutch_Shared_Result_With_Frequency5.xlsx',
 'Turkish_Dutch_Shared_Join_Result_Without_Frequency5.xlsx']

In [38]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [39]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass